In [5]:
# This notebook seres as the code to generate the dataset that will be used to build the Semantic Search Model as well as the Recommender model
# It is best to run this file every new season of anime or once every month to update the anime to the most current
# This data comes from the jikan.moe.api 

# Mount Google Drive (Because i am using Google Colab)
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Lets Load the dataset from jikan.moe.api
import requests
import json
import datetime
import tqdm
import time
import os

# We will take the top anime
api_url = 'https://api.jikan.moe/v4/top/anime'

ourdata = []
# Not All data is needed for the content based recommender system,Thus lets only keep the fields that are neccesary
csvheader = ['MAL ID','Title','Medium Type','Original Source','Status','Rating','Score','ScoredBy','Members','Favourite','Synopsis','Producers','Studios','Genres','Themes','Demographic']

def get_data(page):
  response = requests.get(api_url+f'?page={page}')
  response.raise_for_status()
  data = response.json()
  for x in data['data']:
      # From the jikan docs these are the fields that have a list of items
      genres_name = [y['name'] for y in x['genres']]
      producer_name = [y['name'] for y in x['producers']]
      studios_name = [y['name'] for y in x['studios']]
      themes_name = [y['name'] for y in x['themes']]
      demographics_name = [y['name'] for y in x['demographics']]
      content = [
                x['mal_id'],
                x['title'],
                x['type'],
                x['source'],
                x['status'],
                x['rating'],
                x['score'],
                x['scored_by'],
                x['members'],
                x['favorites'],
                x['synopsis'],
                producer_name if x['producers'] else '',
                studios_name if x['studios'] else '',
                genres_name if x['genres'] else '',
                themes_name if x['themes'] else '',
                demographics_name if x['demographics'] else ''
                ]
      ourdata.append(content)
          
# Lets wait as too many request might overload the API          
wait= 1
def scrape_db():
  last_page = requests.get(api_url).json()['pagination']['last_visible_page']
  print('Started:', datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

  for page in tqdm.trange(1,last_page+1):
    start = time.perf_counter()
    get_data(page)
    end = time.perf_counter()
    time.sleep(max(0, start + wait - end))

  print('Finished:', datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))


# Function to run the code to fetch our data
scrape_db()

Started: 2023-03-10 15:44:43


100%|██████████| 888/888 [14:50<00:00,  1.00s/it]

Finished: 2023-03-10 15:59:33


In [7]:
# This will write the data as well as the csvheader to a csv file,I have saved it under my google drive but feel free to change as per the situation
import csv
with open('/content/drive/MyDrive/Anime_Recommender/animelist.csv','w',encoding = 'UTF8',newline='') as f:
  writer = csv.writer(f)
  writer.writerow(csvheader)
  writer.writerows(ourdata)